In [1]:
import pandas as pd
import numpy as np
import re

/var/folders/1x/jp3vqcs53vv9kxc9f40hgztc0000gn/T/ipykernel_45593/1346524901.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
# Glossary of Comic Terms
# https://comics.ha.com/c/ref/glossary.zx

In [3]:
comics_data_df = pd.read_csv('./data/comics_data.csv')
comics_data_df.head()

,Unnamed: 0,title,title_link,issue_link,cover_link,issue,cover_date,cover_price,current_value,hist_prices_link,...,contributors_names,contributors_roles,characters,pub_id,volume,years,issues_total,pub_name,pub_titles_total,pub_issues_total
0,0,'Mazing Man,/series/4036/0/dc-comics-mazing-man,https://comicbookrealm.com/series/4036/54341/d...,/cover-scan/acb40c8a57d50781194e37be2a94b3d5/l...,# 1,Jan '86,$0.75,$3.00,/comic.php?a=historical&comic=54341?keepThis=t...,...,"['Stephen Destefano', 'Alan Gold', 'Nansi Hool...","['Pencils-CoverArt', 'Editor', 'Colors', 'Cove...","[[""'Mazing Man"", '/character-photos/character_...",1.0,NaN,1986,15.0,DC Comics,4688,57043
1,1,'Mazing Man,/series/4036/0/dc-comics-mazing-man,https://comicbookrealm.com/series/4036/54350/d...,/cover-scan/293f4e231eccb71ad6e6bc3fe3e49c4f/l...,# 10,Oct '86,$0.75,$3.00,/comic.php?a=historical&comic=54350?keepThis=t...,...,no contributors assigned,no contributors assigned,no characters assigned,1.0,NaN,1986,15.0,DC Comics,4688,57043
2,2,'Mazing Man,/series/4036/0/dc-comics-mazing-man,https://comicbookrealm.com/series/4036/54351/d...,/cover-scan/45c7a284efa7d07f7bb5c9afb3080ffd/l...,# 11,Nov '86,$0.75,$3.00,/comic.php?a=historical&comic=54351?keepThis=t...,...,no contributors assigned,no contributors assigned,no characters assigned,1.0,NaN,1986,15.0,DC Comics,4688,57043
3,3,'Mazing Man,/series/4036/0/dc-comics-mazing-man,https://comicbookrealm.com/series/4036/54352/d...,/cover-scan/21764f5da802b25622831976f1619aa4/l...,# 12,Dec '86,$0.75,$4.00,/comic.php?a=historical&comic=54352?keepThis=t...,...,"['Craig Boldman', 'Stephen DeStafano', 'Nansi ...","['Inks', 'Scripts-CoverArt', 'Colors', 'CoverA...","[[""'Mazing Man"", '/character-photos/character_...",1.0,NaN,1986,15.0,DC Comics,4688,57043
4,4,'Mazing Man,/series/4036/0/dc-comics-mazing-man,https://comicbookrealm.com/series/4036/54342/d...,/cover-scan/3a4df503a1fb2450b1227f4c1f0c45d9/l...,# 2,Feb '86,$0.75,$3.00,/comic.php?a=historical&comic=54342?keepThis=t...,...,no contributors assigned,no contributors assigned,no characters assigned,1.0,NaN,1986,15.0,DC Comics,4688,57043


In [4]:
comics_data_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461301 entries, 0 to 461300
Data columns (total 30 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Unnamed: 0          461301 non-null  int64  
 1   title               461301 non-null  object 
 2   title_link          461301 non-null  object 
 3   issue_link          461301 non-null  object 
 4   cover_link          461301 non-null  object 
 5   issue               461301 non-null  object 
 6   cover_date          369712 non-null  object 
 7   cover_price         461301 non-null  object 
 8   current_value       461301 non-null  object 
 9   hist_prices_link    461301 non-null  object 
 10  searched            461301 non-null  int64  
 11  owned               461301 non-null  int64  
 12  pages               208011 non-null  float64
 13  rating              461301 non-null  float64
 14  rating_count        461301 non-null  int64  
 15  ISBN-UPC            197912 non-nul

In [5]:
# extract titles' and issues' unique ids
comics_data_df['title_id'] = comics_data_df.copy().title_link.str.extract(pat=r'/series.(\d+)/\d.+')
comics_data_df['issue_id'] = comics_data_df.copy().issue_link.str.extract(pat=r'/series/\d+/(\d+)/.+')

In [6]:
# keep relevant columns for analysis
df = comics_data_df[['pub_name', 'title', 'title_id', 'issue', 'variant_of', 'volume', 'cover_date', 
           'years', 'cover_price', 'current_value', 'searched', 
          'owned', 'issues_total', 'est_print_run',  
           'pub_titles_total', 'pub_issues_total',
           'contributors_names', 'contributors_roles', 'characters', 'synopsis']]

In [7]:
# turn prices into numerical values
df.loc[:, 'cover_price'] = df.copy().loc[:, 'cover_price'].str.replace('[$,]', '', regex=True)
df.loc[:, 'cover_price'] = df.copy().loc[:, 'cover_price'].str.replace('Free', '0', regex=False)
df.loc[:, 'cover_price'] = df.copy().loc[:, 'cover_price'].apply(eval)

In [8]:
df.loc[:, 'current_value'] = df.copy().loc[:, 'current_value'].str.replace('[$,]', '', regex=True)
df.loc[:, 'current_value'] = df.copy().loc[:, 'current_value'].str.replace('Free', '0', regex=False)
df.loc[:, 'current_value'] = df.copy().loc[:, 'current_value'].apply(eval)

In [9]:
# Many issues with cover price 0 have high current value so we keep them
df[df.cover_price == 0].sort_values('current_value', ascending=False).head(2)

,pub_name,title,title_id,issue,variant_of,volume,cover_date,years,cover_price,current_value,searched,owned,issues_total,est_print_run,pub_titles_total,pub_issues_total,contributors_names,contributors_roles,characters,synopsis
44922,DC Comics,Superman Comics,47267,Ashcan,NaN,One-Shot,NaN,1939,0.0,300000.0,97,1,1.0,NaN,4688,57043,no contributors assigned,no contributors assigned,no characters assigned,"In House Use Only, Not Distributed to Newsstands"
569,DC Comics,Action Comics,2618,# 2,NaN,Vol. 1,Jul '38,1938-2011 2016-Present,0.0,175000.0,18536,7,1269.0,NaN,4688,57043,"['Bernard Baily', 'Russell Cole', 'Sven Elven'...","['Pencils-Inks', 'Scripts-Pencils-Inks', 'Penc...","[['Adam Springer', '/avatars/default.gif', '(N...",2nd appearance of Superman (Clark Kent / Kal-E...


### `issue` column

There are several types of issues as indicated by their numbering and many unconventional numberings:

* Simple ascending numbering # 1-...

* Issues with numbering of the format: # (number)(variant-cover/ special ed./ convention ed. etc.): Normally these issues should be indicated as "variant of" (issue number) in the `variant_of` column

* 'Ashcan' Issues which are traditionally used to promote a new series (see [wiki](https://en.wikipedia.org/wiki/Ashcan_comic)). Ashcan comics can be quite rare and valuable, especially from the [Golden Age](https://en.wikipedia.org/wiki/Golden_Age_of_Comic_Books) (30s-50s)

* Issues with `nn` meaning 'non-numbered'. Typically 0th issues or one-shots

* Of "Vol ** # **" format (where ** indicated a number). i.e. two enumerations are indicated, one for volume, one for issue number

* Instead of issue number a date is given e.g. "Spring 2007" or simply "1989"

* Numbering which includes letters e.g. "C-2" or "x"

* Just the name of the protagonist or in general something explanatory e.g. "Batwoman"

* The print run is indicated e.g. '# 2 - 2nd print'


In [10]:
# make a copy before doing heavy changes
df1 = df.copy()

In [11]:
# get the print number
prints = df1.issue.str.extract('.*(\d{1,2})[a-zA-Z]{2}[\s\-][pP]rint')
df1['print_nr'] = prints.fillna(1).astype(int)
# extract volume and issue information from issue of the type e.g. 'V5-14' or 'V9 6' or 'V12#6'
vols = (df1.issue.str.extractall('.*[Vv][oO]?[lL]?(\d+)[\s\-#]+(\d+)')
            .droplevel(1).rename({0: 'vol', 1: 'issue'}, axis=1))
# extract volume and issue information from issue of the type e.g. 'B-14' or 'C 6'
lett = df1.issue.str.extractall('([A-Z])[#\s\-](\d+)\s?.*').droplevel(1).rename({0: 'vol', 1: 'issue'}, axis=1)

In [12]:
# Find all ashcan issues
idx_ash = df1.issue.str.contains('[Aa]shcan')
# Find all non-numbered and ashcan issues
idx_nn = df1.issue.str.contains('(?:\s+|^)nn-?(?:\s*|$)')
df1.loc[idx_nn, 'volume'] = 'one-shot'
# Find all previews
idx_prev = df1.issue.str.contains('[Pp]review')
# find all issues that contain also the volume information
vols_idx = df1.issue.str.contains('.*[Vv][oO]?[lL]?#?\d+[\s\-#]+\d+')
# find issues that are properly numbered
numb_idx = df1.issue.str.contains('.*#\s*-?\d+\D*.*')
# find issues that are numbered as e.g. B-5 etc.
lett_idx = df1.issue.str.contains('^[A-Z][#\s\-]\d+\s?.*')
# find issues that are numbered by single capital letters
caplet_idx = df1.issue.str.contains('^[A-Z]$')
# find issues that are numbered by single lowercase letters
lowlet_idx = df1.issue.str.contains('^[a-z]$')
# find issues from free comic book day
fcbd_idx = df1.issue.str.contains('fcbd|free comic')
df1.loc[fcbd_idx, 'volume'] = 'fcbd'

# Extract issue numbers by the ones that are numbered in the conventional way
numbered = df1.issue.str.extract(r'#\s*(\d+)\D*')

# give numberical values to non-numbered special-type issues. This is just a convention
# ashcan --> -1
numbered.loc[idx_ash] = str(0)
# nn --> 1
numbered.loc[idx_nn] = str(1)
# preview --> 0.5
numbered.loc[idx_prev] = str(0.5)
# free comic book day (fcbd) --> 1
numbered.loc[fcbd_idx] = str(1)
# single capital letter numbering to int numbers
numbered.loc[caplet_idx] = df1[caplet_idx].issue.apply(lambda x: str(ord(x) - 64)).values.reshape(-1, 1)
# single lower letter numbering to int numbers
numbered.loc[lowlet_idx] = df1[lowlet_idx].issue.apply(lambda x: str(ord(x) - 96)).values.reshape(-1, 1)

# add volume and issue info extracted from issue column
numbered.loc[vols_idx, 0] = vols.issue 
df1.loc[vols_idx, 'volume'] = vols.vol
numbered.loc[lett_idx, 0] = lett.issue
df1.loc[lett_idx, 'volume'] = lett.vol.apply(lambda x: str(ord(x) - 64))

# create new column with the issues numbers
df1.insert(4, 'issue_nr', pd.to_numeric(numbered[0]))

In [13]:
# find all the above
regex = r'(?:\s+|^)nn-?(?:\s*|$)|.*#\s*-?\d+\D*.*|[Aa]shcan|[Pp]review|.*[Vv]\d+\s?-?\s?\d+|[A-Z][\s-]\d+\s?.*|^[A-Z]$|fcbd|free comic|^[a-z]$'
idx_good = (df1.issue.str.contains(regex))

In [14]:
# All that remained that we will drop since issue number is considered essential
print(df1[~idx_good].shape)
print(df1[~idx_good].current_value.sum())

(2003, 22)
62496.0


In [15]:
df2 = df1.dropna(subset=['issue_nr'])
# drop issues with issue_nr > 10000 - considered outliers
df2 = df2[~(df2.issue_nr > 10000)]

### `volume` column

Let's extract important info and clean it

In [16]:
# All none values for volume can be considered volume 1
df2.volume.isna().sum()

247376

In [17]:
# take care of volume numbers formatted as e.g. 'Vol. 5'
vol_nr = df2.volume.str.extract('.*[vV][oO][lL][.\s\-]+(\d+)')
vol_nr_idx = vol_nr.notna()[0]

df2.loc[vol_nr_idx, 'volume'] =  vol_nr

In [18]:
df2.volume.fillna('0', inplace=True)

/var/folders/1x/jp3vqcs53vv9kxc9f40hgztc0000gn/T/ipykernel_45593/1733800711.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df2.volume.fillna('0', inplace=True)


In [19]:
# Volume of type 'one-shot'
oneshot_idx = df2.volume.str.contains('one[\s\-=]+shot', flags=re.IGNORECASE)
df2.loc[oneshot_idx, 'volume'] = 'one-shot'
# Volume of type trade paper-back (TPB) or soft-cover (SC)
tpb_idx = df2.volume.str.contains('TRPB|TPB|TP|paperback|sc|soft', flags=re.IGNORECASE)
df2.loc[tpb_idx, 'volume'] = 'TPB/SC'
# Volume of type Hard Cover (HC)
hc_idx = df2.volume.str.contains('HC|hard', flags=re.IGNORECASE)
df2.loc[hc_idx, 'volume'] = 'HC'
# Volume of type graphic novel (GN)
gn_idx = df2.volume.str.contains('^gn$|^ogn$', flags=re.IGNORECASE)
df2.loc[gn_idx, 'volume'] = 'GN'
# Volume of type mini-series/ maxi-series /limited-series which are ofter used interchangeably (https://en.wikipedia.org/wiki/Limited_series_(comics))
mini_idx = df2.volume.str.contains('mini|maxi|limit', flags=re.IGNORECASE)
df2.loc[mini_idx, 'volume'] = 'limited-series'
# Volume of type magazine
mag_idx = df2.volume.str.contains('magazine', flags=re.IGNORECASE)
df2.loc[mag_idx, 'volume'] = 'magazine'
# Volume of type fanzine
fan_idx = df2.volume.str.contains('fanzine', flags=re.IGNORECASE)
df2.loc[fan_idx, 'volume'] = 'fanzine'
# Volume for fcbd comics
FCBD_idx = df2.volume.str.contains('fcbd', flags=re.IGNORECASE)
df2.loc[FCBD_idx, 'volume'] = 'fcbd'
# Volume for four-colour comics
four_idx = df2.volume.str.contains('(?=.*[fF]our)(?=.*[cC]olor|.*[cC]olour)', flags=re.IGNORECASE)
df2.loc[four_idx, 'volume'] = 'four-color'

In [20]:
volume_cats = ['limited-series', 'one-shot', 'magazine', 'TPB/SC', 'HC', 'GN', 'four-color', 
               'Prepack', 'fcbd', 'fanzine']
volume_nums = list(range(21))
volume_lst = volume_cats + [str(n) for n in volume_nums]

In [21]:
other_idx = ~df2.volume.isin(volume_lst)

In [22]:
# get rid of those, in most cases these are not comic books
df3 = df2[~other_idx]

### `variant_of` column

Turn column into numerical containing the number of the issue this issue is a variant of

In [23]:
# use info in variant to fill volume information
variant = df3[df3.variant_of.notna()]

In [24]:
# find variant number in the form 'Soft Cover or TPB # 213'
tpb_var_idx = variant.variant_of.str.contains('TRPB|TPB|TP|paperback|sc|soft', flags=re.IGNORECASE)
# extract issue number
tpb_var_num = variant[tpb_var_idx].variant_of.str.extract(r'.*#?\s*(\d+)\D*')

# find variant number in the form 'Hard Cover # 213'
hc_var_idx = variant.variant_of.str.contains('hard|HC', flags=re.IGNORECASE)
# extract issue number
hc_var_num = variant[hc_var_idx].variant_of.str.extract(r'.*#?\s*(\d+)\D*')

In [25]:
var_tpb_index = tpb_var_idx[(tpb_var_idx == True)].index
df3.loc[var_tpb_index, 'volume'] = 'TPB/SC'

var_hc_index = tpb_var_idx[(hc_var_idx == True)].index
df3.loc[var_hc_index, 'volume'] = 'HC'

In [26]:
# Make new column where variant is bool True or False
is_variant = df3.variant_of.notna()

In [27]:
df3.loc[:, 'variant'] = is_variant.copy()

/var/folders/1x/jp3vqcs53vv9kxc9f40hgztc0000gn/T/ipykernel_45593/2818492217.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.loc[:, 'variant'] = is_variant.copy()


### `synopsis` column

Try to find other information from the synopsis of each issue e.g. if it is an incentive cover variant. Additional ideas for information extraction from this column are very welcome!

In [28]:
# pd.options.display.max_colwidth = 1000
# pd.options.display.max_rows = 500

In [29]:
df3.synopsis.notna().value_counts()

synopsis
True     289702
False    169202
Name: count, dtype: int64

In [30]:
df3[df3.synopsis.notna()].synopsis.sample(1)

279806    Whitman Reprint
Name: synopsis, dtype: object

In [31]:
incentive_idx = df3.synopsis.str.contains('incentive', flags=re.IGNORECASE).fillna(False)

/var/folders/1x/jp3vqcs53vv9kxc9f40hgztc0000gn/T/ipykernel_45593/2213686341.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  incentive_idx = df3.synopsis.str.contains('incentive', flags=re.IGNORECASE).fillna(False)


In [32]:
limited_idx = df3.synopsis.str.contains('(?=.*limited)(?=.*edition|.*cover|.*variant|.*issue|.*copies|.*copy)', flags=re.IGNORECASE).fillna(False)

/var/folders/1x/jp3vqcs53vv9kxc9f40hgztc0000gn/T/ipykernel_45593/925029850.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  limited_idx = df3.synopsis.str.contains('(?=.*limited)(?=.*edition|.*cover|.*variant|.*issue|.*copies|.*copy)', flags=re.IGNORECASE).fillna(False)


In [33]:
limit_copies = df3[limited_idx].synopsis.str.extract(r'.*\s(\d,?\d+) copies.*|.*limited to (\d,?\d+).*', flags=re.IGNORECASE)

In [38]:
print(limit_copies.columns)

Index([0, 1], dtype='int64')


In [39]:
limit_copies = limit_copies[0].fillna(limit_copies[1])

In [40]:
# deluxe edition (< limited)
deluxe_idx = df3.synopsis.str.contains('(?=.*deluxe)(?=.*edition)', flags=re.IGNORECASE).fillna(False)

/var/folders/1x/jp3vqcs53vv9kxc9f40hgztc0000gn/T/ipykernel_45593/4105790761.py:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  deluxe_idx = df3.synopsis.str.contains('(?=.*deluxe)(?=.*edition)', flags=re.IGNORECASE).fillna(False)


In [41]:
# create a column 'special' to indicate limited/deluxe edition etc
df3.loc[:, 'special'] = 'not'

/var/folders/1x/jp3vqcs53vv9kxc9f40hgztc0000gn/T/ipykernel_45593/855862644.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.loc[:, 'special'] = 'not'


In [42]:
df3.loc[deluxe_idx, 'special'] = 'deluxe'
df3.loc[limited_idx, 'special'] = 'limited'
df3.loc[incentive_idx, 'special'] = 'incentive'

In [43]:
df3.special.value_counts()

special
not          428151
incentive     23510
limited        6644
deluxe          599
Name: count, dtype: int64

In [44]:
# The number of copies for the limited editions wherever it could be obtained
limit_copies = limit_copies.str.replace(',', '').astype(float).dropna()

#### Get first appearance and maybe other special events from `characters` column 

In [45]:
def get_events(char_list):
    
    if char_list == 'no characters assigned':
        return 'nothing'
    else:
        chars = eval(char_list)
        events = []
        for char in chars:
            if char[-1] != '':
                events.append(char[-1])
            else:
                continue
        if events == []:
            return 'nothing'
        else:
            return str(events)
            

In [46]:
events = df3.characters.apply(get_events)

In [47]:
df3.loc[:, 'first_appear_event'] = events.str.contains('First appearance', flags=re.IGNORECASE)
df3.loc[:, 'death_event'] = events.str.contains('Death', flags=re.IGNORECASE)
df3.loc[:, 'origin_event'] = events.str.contains('origin', flags=re.IGNORECASE)

/var/folders/1x/jp3vqcs53vv9kxc9f40hgztc0000gn/T/ipykernel_45593/2525520355.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.loc[:, 'first_appear_event'] = events.str.contains('First appearance', flags=re.IGNORECASE)
/var/folders/1x/jp3vqcs53vv9kxc9f40hgztc0000gn/T/ipykernel_45593/2525520355.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3.loc[:, 'death_event'] = events.str.contains('Death', flags=re.IGNORECASE)
/var/folders/1x/jp3vqcs53vv9kxc9f40hgztc0000gn/T/ipykernel_45593/2525520355.py:3: 

### Prepare final dataset

In [48]:
df4 = df3.copy()

In [49]:
df4.info()

<class 'pandas.core.frame.DataFrame'>
Index: 458904 entries, 0 to 461300
Data columns (total 27 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   pub_name            458904 non-null  object 
 1   title               458904 non-null  object 
 2   title_id            458904 non-null  object 
 3   issue               458904 non-null  object 
 4   issue_nr            458904 non-null  float64
 5   variant_of          131004 non-null  object 
 6   volume              458904 non-null  object 
 7   cover_date          368114 non-null  object 
 8   years               457070 non-null  object 
 9   cover_price         458904 non-null  object 
 10  current_value       458904 non-null  object 
 11  searched            458904 non-null  int64  
 12  owned               458904 non-null  int64  
 13  issues_total        454034 non-null  float64
 14  est_print_run       30311 non-null   object 
 15  pub_titles_total    458904 non-null  in

In [51]:
df4 = df4[['pub_name', 'title', 'title_id', 'issue_nr', 'variant', 'volume', 'print_nr', 
          'cover_date', 'cover_price', 'current_value', 'special', 'searched', 'owned',
         'first_appear_event', 'death_event', 'origin_event', 'issues_total',
         'pub_titles_total', 'pub_issues_total']]

In [52]:
# take a final look before saving
df4.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Index: 458904 entries, 0 to 461300
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   pub_name            458904 non-null  object 
 1   title               458904 non-null  object 
 2   title_id            458904 non-null  object 
 3   issue_nr            458904 non-null  float64
 4   variant             458904 non-null  bool   
 5   volume              458904 non-null  object 
 6   print_nr            458904 non-null  int64  
 7   cover_date          368114 non-null  object 
 8   cover_price         458904 non-null  object 
 9   current_value       458904 non-null  object 
 10  special             458904 non-null  object 
 11  searched            458904 non-null  int64  
 12  owned               458904 non-null  int64  
 13  first_appear_event  458904 non-null  bool   
 14  death_event         458904 non-null  bool   
 15  origin_event        458904 non-null  bo

In [53]:
# save the cleaned data
df4.to_csv('data_clean.csv')